#  <center> Speech Emotion Recognition <center>

# Importing Libraries

In [5]:
import pandas as pd
import numpy as np

import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data Preparation

In [6]:
# Paths for data.
dataset_path = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"

### Feature Extraction

In [7]:
# Adding 3 types of data augmentation techniques
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(audio_data):
    return librosa.effects.time_stretch(audio_data, rate=0.8)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

In [8]:
# Function to Load data
def load_data(file_path):
    audio, sampling_rate = librosa.load(file_path, duration=2.5, offset=0.6)
    return audio

In [9]:
# Function to preprocess the data
def input_label_define(dataset_path):
    y=[]
    X=[]
    gender=[]
    actors = []
    for folder in os.listdir(dataset_path):
        for file in os.listdir(os.path.join(dataset_path, folder)):
            if file.endswith('.wav'): 
                emotion = file.split('-')[2]
                if int(emotion) == 1:
                    label = 'Neutral'
                elif int(emotion) == 2:
                    label = 'Calm'
                elif int(emotion) == 3:
                    label = 'Happy'
                elif int(emotion) == 4:
                    label = 'Sad'
                elif int(emotion) == 5:
                    label = 'Angry'
                elif int(emotion) == 6:
                    label = 'Fearful'
                elif int(emotion) == 7:
                    label = 'Disgust'
                elif int(emotion) == 8:
                    label = 'Surprised'
                else:
                    label = 'UNK'
                actor = int(file.split('-')[-1].split('.')[0])
                         
                file_path = os.path.join(dataset_path, folder, file)
                audio_array=load_data(file_path)
                
                # gender
                gender_value=file_path.split("-")[-1].split(".")[0]
                if (int(gender_value)%2)==0:
                    gender_feature=0
                else:
                    gender_feature=1
                
                X.append(load_data(file_path))
                y.append(label)
                gender.append(gender_feature)
                actors.append(actor)
                
                # data with noise
                X.append(noise(audio_array))
                y.append(label)
                gender.append(gender_feature)
                actors.append(actor)
                
                # Data with shift
                X.append(shift(audio_array))
                y.append(label)
                gender.append(gender_feature)
                actors.append(actor)
                
                # data with stretching and pitching
                X.append(stretch(audio_array))
                y.append(label)
                gender.append(gender_feature)
                actors.append(actor)
            
    return X,y,gender, actors

In [10]:
X_r_aug, y_r_aug, gender, actors= input_label_define(dataset_path)

In [9]:
np.array(X_r_aug).shape,np.array(y_r_aug).shape,np.array(y_r_aug).shape

((5760,), (5760,), (5760,))

In [ ]:
# Saving the raw input data
# import pickle

# open a file in binary write mode
# with open('RAVDESS_raw_aug.pkl', 'wb') as f:
    # write the array to the file using pickle.dump()
#     pickle.dump(np.array(X_r_aug), f)

### Extract Accoustic features

In [10]:
def acoustic_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

In [11]:
sampling_rate=22050

In [ ]:
X_ac = []
for i in X_r_aug:
    X_ac.append(acoustic_features(i))

In [ ]:
y_ac=y_r_aug
np.array(X_ac).shape,np.array(y_ac).shape

((5760, 162), (5760,))

In [ ]:
# Accoustic features saved
# np.save('RAVDESS_Aug_Acoustic.npy',np.array(X_ac))
# np.save('RAVDESS_Aug_Acoustic_emotion.npy',np.array(y_ac))

### Extract Statistical Features

In [ ]:
import os, glob
import librosa
import numpy as np
import scipy

# Function to extract features from each audio file
def statistical_features(audio):
    mean = np.mean(audio)
    variance = np.var(audio)
    skewness = scipy.stats.skew(audio)
    kurtosis = scipy.stats.kurtosis(audio)
    audio_rms = librosa.feature.rms(y=audio)
    audio_rms_mean = audio_rms.mean()
    spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sampling_rate)[0]
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sampling_rate)[0]
    mfccs = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=13)
    mfcc = mfccs.mean()
    return [mean, variance, skewness, kurtosis, audio_rms_mean, spectral_centroids.mean(), spectral_bandwidth.mean(), mfcc]
  

In [ ]:
X_stat = []
for i in X_r_aug:
    X_stat.append(statistical_features(i))
np.array(X_stat).shape

(5760, 8)

In [ ]:
# Saving statistical features
# np.save('RAVDESS_Aug_Stat.npy',np.array(X_stat))

In [11]:
X_stat = np.load('/kaggle/input/ravdess-revathi-npy/RAVDESS_Aug_Stat.npy')
X_ac = np.load('/kaggle/input/ravdess-revathi-npy/RAVDESS_Aug_Acoustic.npy')
y_ac = np.load('/kaggle/input/ravdess-revathi-npy/RAVDESS_Augmented_emotion.npy')

In [12]:
Features = pd.DataFrame(X_ac)
Features['labels'] = y_ac
Features.to_csv('features.csv', index=False)
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.237382,0.682273,0.670519,0.614793,0.579039,0.566803,0.637206,0.667451,0.663883,0.669903,...,0.000234,0.000235,0.000138,0.000128,0.000299,0.000360,0.000275,0.000173,2.896729e-05,Surprised
1,0.331642,0.727662,0.734769,0.731634,0.710158,0.717446,0.743283,0.713234,0.700561,0.683346,...,0.001221,0.001236,0.001138,0.001159,0.001292,0.001365,0.001208,0.001139,1.052731e-03,Surprised
2,0.330164,0.730809,0.734809,0.727598,0.707369,0.717330,0.745340,0.724385,0.699645,0.686211,...,0.001076,0.001074,0.000989,0.001015,0.001213,0.001241,0.001193,0.001121,8.812787e-04,Surprised
3,0.329427,0.728282,0.734317,0.737076,0.720081,0.723116,0.740559,0.714165,0.689678,0.682365,...,0.001137,0.001108,0.001044,0.001099,0.001215,0.001272,0.001126,0.001112,9.334161e-04,Surprised
4,0.256113,0.621621,0.580278,0.598515,0.628296,0.598015,0.597272,0.650352,0.698636,0.681863,...,0.000018,0.000016,0.000013,0.000012,0.000007,0.000008,0.000008,0.000005,5.802853e-07,Neutral


In [13]:
Features.shape

(5760, 163)

In [14]:
# Combining both statistic and deep features and gender to create a common dataframe
stat_col_index=len(Features.columns)
stat_features=8
X_stat_df = pd.DataFrame(X_stat,columns=list(range(stat_col_index,stat_col_index+stat_features)))

gender_col_index=len(Features.columns)+stat_features
X_gender_df=pd.DataFrame(gender,columns=list(range(gender_col_index,gender_col_index+1)))
# stacking horizontally for getting all combinations
X_ac_stat_nogen = pd.concat([Features, X_stat_df], axis=1)

X_ac_stat = pd.concat([Features, X_stat_df,X_gender_df], axis=1)

In [15]:
np.array(X_ac_stat).shape

(5760, 172)

## Data Preparation

In [16]:
X = X_ac_stat.drop('labels',axis=1).values
Y = X_ac_stat['labels'].values

In [17]:
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [18]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=0, shuffle=True,stratify=Y)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4608, 171), (4608, 8), (1152, 171), (1152, 8))

In [19]:
# Normalizing our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4608, 171), (4608, 8), (1152, 171), (1152, 8))

In [20]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4608, 171, 1), (4608, 8), (1152, 171, 1), (1152, 8))

In [21]:
# Acoustic feature segregation for deep feature extraction
x_train_ac = x_train[:, :162]
x_test_ac=x_test[:, :162]

x_train_stat = x_train[:, 162:170]
x_test_stat=x_test[:, 162:170]

x_train_gen=x_train[:,-1]
x_test_gen=x_test[:,-1]

In [22]:
x_train_ac.shape,x_train_stat.shape,x_train_gen.shape

((4608, 162, 1), (4608, 8, 1), (4608, 1))

In [23]:
x_train_stat.shape,x_test_stat.shape,x_test_gen.shape

((4608, 8, 1), (1152, 8, 1), (1152, 1))

In [24]:
x_train_gen.shape,x_test_gen.shape

((4608, 1), (1152, 1))

## Modelling

In [25]:
model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train_ac.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=8, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 162, 256)          1536      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 81, 256)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 20736)             0         
                                                                 
 dense (Dense)               (None, 32)                663584    
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                        

In [26]:
model.input

<KerasTensor: shape=(None, 162, 1) dtype=float32 (created by layer 'conv1d_input')>

In [27]:
x_train_ac.shape,y_train.shape

((4608, 162, 1), (4608, 8))

In [28]:
x_test_ac.shape,y_test.shape

((1152, 162, 1), (1152, 8))

In [29]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
history=model.fit(x_train_ac, y_train, batch_size=64, epochs=50, validation_data=(x_test_ac, y_test), callbacks=[rlrp])

Epoch 1/50
72/72 [==============================] - 4s 45ms/step - loss: 1.8179 - accuracy: 0.2934 - val_loss: 1.6522 - val_accuracy: 0.3976 - lr: 0.0010
Epoch 2/50
72/72 [==============================] - 3s 47ms/step - loss: 1.6517 - accuracy: 0.3672 - val_loss: 1.5590 - val_accuracy: 0.4366 - lr: 0.0010
Epoch 3/50
72/72 [==============================] - 4s 51ms/step - loss: 1.5477 - accuracy: 0.4141 - val_loss: 1.4257 - val_accuracy: 0.4852 - lr: 0.0010
Epoch 4/50
72/72 [==============================] - 4s 52ms/step - loss: 1.4924 - accuracy: 0.4440 - val_loss: 1.4083 - val_accuracy: 0.4905 - lr: 0.0010
Epoch 5/50
72/72 [==============================] - 3s 44ms/step - loss: 1.4242 - accuracy: 0.4627 - val_loss: 1.3350 - val_accuracy: 0.5165 - lr: 0.0010
Epoch 6/50
72/72 [==============================] - 3s 46ms/step - loss: 1.3381 - accuracy: 0.4963 - val_loss: 1.2648 - val_accuracy: 0.5347 - lr: 0.0010
Epoch 7/50
72/72 [==============================] - 3s 40ms/step - loss: 1.3

In [33]:
# predicting on test data.
pred_test = model.predict(x_test_ac)
# y_pred = encoder.inverse_transform(pred_test)

# y_test = encoder.inverse_transform(y_test)
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_test, pred_test)

36/36 [==============================] - 0s 6ms/step


In [34]:
model.evaluate(x_test_ac,y_test)

36/36 [==============================] - 0s 7ms/step - loss: 0.6442 - accuracy: 0.7674


[0.644156277179718, 0.7673611044883728]

### Getting the features from the Dense layer of conv1d model

In [35]:
model.layers[-3]

In [36]:
from tensorflow.keras.models import Model

conv1d_feature_model=Model(inputs=model.input,outputs=model.layers[-3].output)

In [37]:
conv1d_features=conv1d_feature_model.predict(x_train_ac)

144/144 [==============================] - 1s 6ms/step


In [38]:
conv1d_features_test=conv1d_feature_model.predict(x_test_ac)

36/36 [==============================] - 0s 7ms/step


### Concatenating with statistical features

In [ ]:
x_train_gen.shape,y_train.shape

((4608, 1), (4608, 8))

In [ ]:
x_train_stat.shape

(4608, 8, 1)

In [ ]:
# Getting back old shape of xtrain and xtest
x_train_stat = np.squeeze(x_train_stat)

# x_train_gen = np.squeeze(x_train_gen)
x_train_stat.shape,y_train.shape,x_train_gen.shape

((4608, 8), (4608, 8), (4608, 1))

In [ ]:
conv1d_features.shape,x_train_stat.shape,x_train_gen.shape

((4608, 32), (4608, 8), (4608, 1))

In [ ]:
x_train_concat=np.concatenate((conv1d_features,x_train_stat,x_train_gen),axis=1)

In [ ]:
x_train_concat.shape

(4608, 41)

In [ ]:
x_test_stat = np.squeeze(x_test_stat)

In [ ]:
conv1d_features_test.shape,x_test_stat.shape,x_test_gen.shape

((1152, 32), (1152, 8), (1152, 1))

In [ ]:
x_test_concat=np.concatenate((conv1d_features_test,x_test_stat,x_test_gen),axis=1)

In [ ]:
x_test_concat.shape

(1152, 41)

## 1) Output of extracted features to Machine Learning Classification model

### Classification using Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

In [ ]:
rfc.fit(x_train_concat, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [ ]:
y_pred = rfc.predict(x_test_concat)

In [ ]:
y_pred.shape,y_test.shape

((1152, 8), (1152, 8))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

0.7196180555555556

### Just based on Deep Learning Features

In [ ]:
rfc.fit(conv1d_features, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [ ]:
y_pred = rfc.predict(conv1d_features_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

0.7213541666666666

### Just based on Statistical Features

In [ ]:
rfc.fit(x_train_stat, y_train)

In [ ]:
y_pred = rfc.predict(x_test_stat)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

## 2) Extracted features to Deep Learning model 

### Just Based on Deep Features

In [ ]:
pd.DataFrame(conv1d_features)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.000000,2.411026,0.0,2.298269,0.0,0.0,2.211152,0.0,3.722932,0.000000,...,0.222004,4.571744,0.000000,0.0,0.0,0.000000,0.0,1.423667,0.000000,0.000000
1,4.174733,0.481925,0.0,1.357758,0.0,0.0,0.000000,0.0,2.500226,1.420972,...,0.000000,2.333144,0.000000,0.0,0.0,0.000000,0.0,1.353853,0.332708,0.000000
2,3.840408,0.147967,0.0,0.000000,0.0,0.0,0.000000,0.0,0.434528,1.629446,...,0.244161,0.000000,0.000000,0.0,0.0,0.000000,0.0,1.246616,3.061129,2.016101
3,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.124334,2.250418,...,0.000000,0.000000,0.000000,0.0,0.0,4.691520,0.0,2.616085,0.000000,0.000000
4,0.000000,0.208451,0.0,8.922934,0.0,0.0,0.000000,0.0,0.000000,2.805537,...,0.000000,0.000000,0.691420,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,0.807749,0.977776,0.0,0.000000,0.0,0.0,4.784795,0.0,4.984905,0.000000,...,2.489061,5.809880,3.307879,0.0,0.0,3.433646,0.0,0.000000,0.000000,0.000000
4604,2.643344,4.106495,0.0,0.000000,0.0,0.0,2.199107,0.0,0.000000,0.000000,...,4.900318,5.265594,1.676853,0.0,0.0,0.000000,0.0,0.000000,0.000000,5.008558
4605,0.424827,0.000000,0.0,0.000000,0.0,0.0,0.875379,0.0,0.000000,0.000000,...,0.046438,0.000000,0.504726,0.0,0.0,3.068745,0.0,0.000000,0.000000,0.000000
4606,0.000000,0.000000,0.0,3.757712,0.0,0.0,3.422144,0.0,0.000000,1.318160,...,1.589476,0.000000,3.584829,0.0,0.0,0.000000,0.0,0.070484,0.000000,0.000000


In [ ]:
conv1d_features.shape,y_train.shape

((4608, 32), (4608, 8))

In [ ]:
model_classify = Sequential()
model_classify.add(Dense(12, input_shape=(32,), activation='relu'))
model_classify.add(Dense(8, activation='softmax'))
model_classify.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [ ]:
model_classify.fit(conv1d_features,y_train,validation_split=0.2,epochs=50)

Epoch 1/50
116/116 [==============================] - 0s 3ms/step - loss: 1.4765 - accuracy: 0.5157 - val_loss: 1.2056 - val_accuracy: 0.6204
Epoch 2/50
116/116 [==============================] - 0s 3ms/step - loss: 1.0556 - accuracy: 0.6712 - val_loss: 0.8966 - val_accuracy: 0.7364
Epoch 3/50
116/116 [==============================] - 0s 3ms/step - loss: 0.7902 - accuracy: 0.7775 - val_loss: 0.6899 - val_accuracy: 0.8091
Epoch 4/50
116/116 [==============================] - 0s 3ms/step - loss: 0.6180 - accuracy: 0.8247 - val_loss: 0.5655 - val_accuracy: 0.8243
Epoch 5/50
116/116 [==============================] - 0s 3ms/step - loss: 0.5129 - accuracy: 0.8557 - val_loss: 0.4863 - val_accuracy: 0.8482
Epoch 6/50
116/116 [==============================] - 0s 3ms/step - loss: 0.4397 - accuracy: 0.8728 - val_loss: 0.4257 - val_accuracy: 0.8644
Epoch 7/50
116/116 [==============================] - 0s 3ms/step - loss: 0.3832 - accuracy: 0.8912 - val_loss: 0.3828 - val_accuracy: 0.8883
Epoch 

### Based on Statistic and Deep Features

In [58]:
x_test_stat.shape

(1152, 8)

In [ ]:
stat_deep.shape

(4608, 40)

In [ ]:
model_classify = Sequential()
model_classify.add(Dense(12, input_shape=(40,), activation='relu'))
model_classify.add(Dense(8, activation='softmax'))
model_classify.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model_classify.fit(stat_deep,y_train,validation_split=0.2,epochs=50)

Epoch 1/50
116/116 [==============================] - 1s 4ms/step - loss: 2.4467 - accuracy: 0.1994 - val_loss: 1.7715 - val_accuracy: 0.3449
Epoch 2/50
116/116 [==============================] - 0s 2ms/step - loss: 1.4959 - accuracy: 0.4704 - val_loss: 1.2647 - val_accuracy: 0.6041
Epoch 3/50
116/116 [==============================] - 0s 3ms/step - loss: 1.0602 - accuracy: 0.6910 - val_loss: 0.9034 - val_accuracy: 0.7592
Epoch 4/50
116/116 [==============================] - 0s 2ms/step - loss: 0.7512 - accuracy: 0.8052 - val_loss: 0.6637 - val_accuracy: 0.8113
Epoch 5/50
116/116 [==============================] - 0s 2ms/step - loss: 0.5711 - accuracy: 0.8481 - val_loss: 0.5292 - val_accuracy: 0.8525
Epoch 6/50
116/116 [==============================] - 0s 2ms/step - loss: 0.4650 - accuracy: 0.8728 - val_loss: 0.4521 - val_accuracy: 0.8623
Epoch 7/50
116/116 [==============================] - 0s 2ms/step - loss: 0.3987 - accuracy: 0.8915 - val_loss: 0.3997 - val_accuracy: 0.8796
Epoch 

## Cross Validation

In [42]:
# Experimenting by reducing the learning rate
from keras.optimizers import Adam

opt = keras.optimizers.Adam(learning_rate=0.0001)

In [40]:
x_train_stat = np.squeeze(x_train_stat)
x_test_stat = np.squeeze(x_test_stat)
stat_deep=np.concatenate((conv1d_features,x_train_stat,x_train_gen),axis=1)
stat_deep_test = np.concatenate((conv1d_features_test,x_test_stat,x_test_gen),axis=1)

In [41]:
X_final = np.concatenate((stat_deep, stat_deep_test), axis=0)
y_final = np.concatenate((y_train, y_test), axis=0)

In [43]:
final_df = pd.DataFrame({'X':list(X_final), 'y':list(y_final), 'actor':actors})
final_df.head()

,X,y,actor
0,"[0.0, 0.0, 0.0, 0.0, 3.582874059677124, 0.0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2
1,"[0.0, 0.0, 0.34155717492103577, 0.0, 4.5268797...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",2
2,"[0.0, 0.0, 0.0705176442861557, 0.0, 5.00018405...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2
3,"[0.0, 0.0, 0.0, 0.0, 1.5666661262512207, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]",2


In [44]:
def create_classification_model():
    model = Sequential()
    model.add(Dense(12, input_shape=(41,), activation='relu'))
    model.add(Dense(8, activation='softmax'))
    return model

In [45]:
def train_evaluate(model, X_train, y_train, X_val, y_val):
    
    model.compile(optimizer = opt , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    model.fit(X_train, y_train, epochs=200, validation_data=(X_val, y_val), verbose=0)
    model.evaluate(X_val, y_val)
    
    return model.evaluate(X_val, y_val)

In [46]:
actors_in_fold = []
train_folds = []
test_fold = []
for i in range(6):
    actors_in_fold.append(list(range((4*i)+1, (4*i)+5)))
    all_folds = list(range(6))
    all_folds.remove(i)
    train_folds.append(all_folds)
    test_fold.append(i)

In [50]:
print('actors_in_fold')
print(actors_in_fold)
print('train_folds')
print(train_folds)
print('test_folds')
print(test_fold)

actors_in_fold
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]
train_folds
[[1, 2, 3, 4, 5], [0, 2, 3, 4, 5], [0, 1, 3, 4, 5], [0, 1, 2, 4, 5], [0, 1, 2, 3, 5], [0, 1, 2, 3, 4]]
test_folds
[0, 1, 2, 3, 4, 5]


In [49]:
cross_valiadate_actors(final_df)

Fold 0/6
30/30 [==============================] - 0s 2ms/step - loss: 0.4162 - accuracy: 0.8646
Done
Fold 1/6
30/30 [==============================] - 0s 2ms/step - loss: 0.4065 - accuracy: 0.8562
Done
Fold 2/6
30/30 [==============================] - 0s 2ms/step - loss: 0.3361 - accuracy: 0.8677
Done
Fold 3/6
30/30 [==============================] - 0s 3ms/step - loss: 0.4528 - accuracy: 0.8406
Done
Fold 4/6
30/30 [==============================] - 0s 1ms/step - loss: 0.4316 - accuracy: 0.8375
Done
Fold 5/6
30/30 [==============================] - 0s 2ms/step - loss: 0.3363 - accuracy: 0.8750
Done


In [ ]:
model_classify = Sequential()
model_classify.add(Dense(12, input_shape=(40,), activation='relu'))
model_classify.add(Dense(8, activation='softmax'))
model_classify.compile(optimizer = opt , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model_classify.fit(stat_deep,y_train,validation_split=0.2,epochs=200)

Epoch 1/200
116/116 [==============================] - 1s 4ms/step - loss: 3.2908 - accuracy: 0.1527 - val_loss: 2.5117 - val_accuracy: 0.1746
Epoch 2/200
116/116 [==============================] - 0s 3ms/step - loss: 2.1879 - accuracy: 0.2138 - val_loss: 1.9020 - val_accuracy: 0.2646
Epoch 3/200
116/116 [==============================] - 0s 3ms/step - loss: 1.8047 - accuracy: 0.3095 - val_loss: 1.6919 - val_accuracy: 0.3623
Epoch 4/200
116/116 [==============================] - 0s 3ms/step - loss: 1.6184 - accuracy: 0.3823 - val_loss: 1.5674 - val_accuracy: 0.4393
Epoch 5/200
116/116 [==============================] - 0s 3ms/step - loss: 1.4943 - accuracy: 0.4533 - val_loss: 1.4740 - val_accuracy: 0.4935
Epoch 6/200
116/116 [==============================] - 0s 2ms/step - loss: 1.4028 - accuracy: 0.4957 - val_loss: 1.3963 - val_accuracy: 0.5239
Epoch 7/200
116/116 [==============================] - 0s 3ms/step - loss: 1.3284 - accuracy: 0.5315 - val_loss: 1.3292 - val_accuracy: 0.5499